In [2]:
import pandas as pd
import numpy as np

C:\Users\SuEv-WS\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\SuEv-WS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\SuEv-WS\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [1]:
con = "postgresql://robot-startml-ro:пароль@""postgres.lab.karpov.courses:6432/startml"

In [11]:
#FEED_DATA

df = pd.read_sql(
    """SELECT * FROM "feed_data"
    WHERE feed_data.action = 'view' LIMIT 100000""",
    con=con
)

In [325]:
df

,timestamp,user_id,post_id,action,target
0,2021-11-06 22:18:07,73646,2150,view,0
1,2021-11-06 22:19:45,73646,3781,view,0
2,2021-11-14 06:20:45,73646,2524,view,0
3,2021-11-14 06:23:12,73646,6757,view,0
4,2021-11-14 06:25:37,73646,7191,view,0
...,...,...,...,...,...
9999995,2021-10-20 22:07:47,157803,4124,view,0
9999996,2021-10-20 22:09:32,157803,4757,view,0
9999997,2021-10-20 22:11:55,157803,881,view,0
9999998,2021-10-20 22:13:28,157803,3427,view,0


# USER DATA

In [4]:
#USER_DATA
user_data = pd.read_sql(
    """SELECT * FROM "user_data" """,
    con=con
)

In [6]:
def age_cat(row):
    teenager = row['age'] < 18
    adolescent = 18 <= row['age'] < 25
    young = 25 <= row['age'] < 35
    grown_up = 35 <= row['age'] < 50
    old = row['age'] >= 50 

    if teenager:
        return 'teenager'
    elif adolescent:
        return 'adolescent'
    elif young:
        return 'young'
    elif grown_up:
        return 'grown_up'
    elif old:
        return 'old'
    else:
        return 'unknown'

user_data = user_data.assign(age=user_data.apply(age_cat, axis=1))

In [7]:
user_data

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,young,Russia,Degtyarsk,3,Android,ads
1,201,0,grown_up,Russia,Abakan,0,Android,ads
2,202,1,teenager,Russia,Smolensk,4,Android,ads
3,203,0,adolescent,Russia,Moscow,1,iOS,ads
4,204,0,grown_up,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,grown_up,Russia,Kaliningrad,4,Android,organic
163201,168549,0,adolescent,Russia,Tula,2,Android,organic
163202,168550,1,grown_up,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,grown_up,Russia,Moscow,3,iOS,organic


# POST_TEXT_DF

In [5]:
#POST_TEXT_DF

post_text_df = pd.read_sql(
    """SELECT * FROM "post_text_df" """,
    con=con
)

In [8]:
post_text_df = post_text_df.drop('text', axis=1)

In [9]:
post_text_df

,post_id,topic
0,1,business
1,2,business
2,3,business
3,4,business
4,5,business
...,...,...
7018,7315,movie
7019,7316,movie
7020,7317,movie
7021,7318,movie


In [12]:
df = pd.merge(user_data, df , on='user_id', how='right')
df = df.sort_values("timestamp").drop(['action'], axis=1)
df = pd.merge(df, post_text_df, on='post_id', how='left')

In [13]:
category_columns = df.loc[:,df.dtypes=='object'].columns.values
category_columns = np.append(category_columns, 'exp_group')

In [14]:
df

,user_id,gender,age,country,city,exp_group,os,source,timestamp,post_id,target,topic
0,9484,0,teenager,Kazakhstan,Nur-Sultan,0,Android,ads,2021-10-01 06:08:40,752,0,entertainment
1,9484,0,teenager,Kazakhstan,Nur-Sultan,0,Android,ads,2021-10-01 06:10:02,1177,0,politics
2,9484,0,teenager,Kazakhstan,Nur-Sultan,0,Android,ads,2021-10-01 06:12:04,5590,0,movie
3,9484,0,teenager,Kazakhstan,Nur-Sultan,0,Android,ads,2021-10-01 06:14:00,1341,1,politics
4,9484,0,teenager,Kazakhstan,Nur-Sultan,0,Android,ads,2021-10-01 06:14:52,1429,0,sport
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,158259,1,adolescent,Russia,Lyubertsy,2,Android,organic,2021-12-29 23:21:46,2375,0,covid
99996,158259,1,adolescent,Russia,Lyubertsy,2,Android,organic,2021-12-29 23:23:16,6506,0,movie
99997,158259,1,adolescent,Russia,Lyubertsy,2,Android,organic,2021-12-29 23:24:40,2757,0,covid
99998,158259,1,adolescent,Russia,Lyubertsy,2,Android,organic,2021-12-29 23:26:29,1841,0,sport


In [15]:
# Учитываем лайки к сегодняшнему дню
true_target = (df.groupby(['user_id', 'post_id']).sum() % 2).target
df = df.drop(['target', 'timestamp'], axis=1)
df = pd.merge(df, true_target, on=('user_id', 'post_id'), how='left')
df = df.drop_duplicates()

In [16]:
df1= df.copy()
df.shape

(95618, 11)

In [17]:
df = df1.copy()

# Делим на трейн и тест

In [18]:
train_df = df.iloc[:-2384141].copy()
test_df = df.iloc[:2384141].copy()

In [19]:
train_df.shape[0] + test_df.shape[0], df.shape[0]

(95618, 95618)

In [20]:
train_df = train_df.set_index(['user_id', 'post_id'])
test_df = test_df.set_index(['user_id', 'post_id'])

In [21]:
train_df

,,gender,age,country,city,exp_group,os,source,topic,target
user_id,post_id,,,,,,,,,


In [22]:
X_train = train_df.drop('target', axis=1)
X_test = test_df.drop('target', axis=1)

y_train = train_df['target']
y_test = test_df['target']

In [339]:
category_columns

array(['age', 'country', 'city', 'os', 'source', 'topic', 'exp_group'],
      dtype=object)

In [340]:
#cbm = CatBoostClassifier()
#cbm.fit(X_train, y_train, cat_features=category_columns)
#cbm.save_model('model', format="cbm")

Learning rate set to 0.455812
0:	learn: 0.4400023	total: 3.99s	remaining: 1h 6m 23s
1:	learn: 0.3806474	total: 7.58s	remaining: 1h 3m 1s
2:	learn: 0.3582878	total: 11.2s	remaining: 1h 2m 14s
3:	learn: 0.3525202	total: 14.4s	remaining: 59m 46s
4:	learn: 0.3508496	total: 16.8s	remaining: 55m 42s
5:	learn: 0.3498233	total: 20.4s	remaining: 56m 15s
6:	learn: 0.3491544	total: 23.5s	remaining: 55m 39s
7:	learn: 0.3487447	total: 27.1s	remaining: 56m 2s
8:	learn: 0.3485296	total: 31.1s	remaining: 57m 8s
9:	learn: 0.3484700	total: 34.6s	remaining: 57m 4s
10:	learn: 0.3482469	total: 37.9s	remaining: 56m 47s
11:	learn: 0.3481783	total: 42s	remaining: 57m 35s
12:	learn: 0.3480698	total: 45.4s	remaining: 57m 30s
13:	learn: 0.3480292	total: 49.7s	remaining: 58m 21s
14:	learn: 0.3479904	total: 52.7s	remaining: 57m 40s
15:	learn: 0.3479669	total: 55.5s	remaining: 56m 51s
16:	learn: 0.3479413	total: 58.4s	remaining: 56m 18s
17:	learn: 0.3479140	total: 1m 2s	remaining: 57m 3s
18:	learn: 0.3478974	total:

152:	learn: 0.3473336	total: 7m 50s	remaining: 43m 25s
153:	learn: 0.3473266	total: 7m 54s	remaining: 43m 25s
154:	learn: 0.3473253	total: 7m 56s	remaining: 43m 20s
155:	learn: 0.3473238	total: 7m 59s	remaining: 43m 13s
156:	learn: 0.3473192	total: 8m 2s	remaining: 43m 10s
157:	learn: 0.3473156	total: 8m 5s	remaining: 43m 8s
158:	learn: 0.3473146	total: 8m 8s	remaining: 43m 3s
159:	learn: 0.3473132	total: 8m 11s	remaining: 42m 58s
160:	learn: 0.3473124	total: 8m 13s	remaining: 42m 53s
161:	learn: 0.3473102	total: 8m 16s	remaining: 42m 50s
162:	learn: 0.3473073	total: 8m 19s	remaining: 42m 46s
163:	learn: 0.3473050	total: 8m 22s	remaining: 42m 43s
164:	learn: 0.3473024	total: 8m 25s	remaining: 42m 39s
165:	learn: 0.3472992	total: 8m 28s	remaining: 42m 35s
166:	learn: 0.3472988	total: 8m 31s	remaining: 42m 30s
167:	learn: 0.3472978	total: 8m 34s	remaining: 42m 26s
168:	learn: 0.3472965	total: 8m 36s	remaining: 42m 22s
169:	learn: 0.3472954	total: 8m 39s	remaining: 42m 18s
170:	learn: 0.3

301:	learn: 0.3471038	total: 15m 3s	remaining: 34m 48s
302:	learn: 0.3471028	total: 15m 6s	remaining: 34m 44s
303:	learn: 0.3471016	total: 15m 8s	remaining: 34m 41s
304:	learn: 0.3470998	total: 15m 11s	remaining: 34m 37s
305:	learn: 0.3470983	total: 15m 14s	remaining: 34m 34s
306:	learn: 0.3470977	total: 15m 17s	remaining: 34m 30s
307:	learn: 0.3470966	total: 15m 20s	remaining: 34m 27s
308:	learn: 0.3470956	total: 15m 22s	remaining: 34m 23s
309:	learn: 0.3470948	total: 15m 25s	remaining: 34m 20s
310:	learn: 0.3470927	total: 15m 28s	remaining: 34m 17s
311:	learn: 0.3470922	total: 15m 31s	remaining: 34m 14s
312:	learn: 0.3470906	total: 15m 34s	remaining: 34m 11s
313:	learn: 0.3470899	total: 15m 37s	remaining: 34m 9s
314:	learn: 0.3470880	total: 15m 41s	remaining: 34m 6s
315:	learn: 0.3470857	total: 15m 44s	remaining: 34m 3s
316:	learn: 0.3470851	total: 15m 47s	remaining: 34m
317:	learn: 0.3470849	total: 15m 49s	remaining: 33m 57s
318:	learn: 0.3470841	total: 15m 52s	remaining: 33m 54s
31

449:	learn: 0.3469000	total: 22m 38s	remaining: 27m 40s
450:	learn: 0.3468991	total: 22m 41s	remaining: 27m 37s
451:	learn: 0.3468978	total: 22m 44s	remaining: 27m 34s
452:	learn: 0.3468965	total: 22m 47s	remaining: 27m 30s
453:	learn: 0.3468950	total: 22m 50s	remaining: 27m 27s
454:	learn: 0.3468941	total: 22m 52s	remaining: 27m 24s
455:	learn: 0.3468939	total: 22m 55s	remaining: 27m 20s
456:	learn: 0.3468937	total: 22m 57s	remaining: 27m 16s
457:	learn: 0.3468923	total: 23m	remaining: 27m 13s
458:	learn: 0.3468916	total: 23m 2s	remaining: 27m 9s
459:	learn: 0.3468905	total: 23m 5s	remaining: 27m 6s
460:	learn: 0.3468869	total: 23m 8s	remaining: 27m 3s
461:	learn: 0.3468865	total: 23m 10s	remaining: 26m 59s
462:	learn: 0.3468847	total: 23m 14s	remaining: 26m 56s
463:	learn: 0.3468823	total: 23m 16s	remaining: 26m 53s
464:	learn: 0.3468797	total: 23m 19s	remaining: 26m 50s
465:	learn: 0.3468787	total: 23m 22s	remaining: 26m 47s
466:	learn: 0.3468783	total: 23m 25s	remaining: 26m 43s
46

597:	learn: 0.3467368	total: 30m 7s	remaining: 20m 15s
598:	learn: 0.3467359	total: 30m 10s	remaining: 20m 12s
599:	learn: 0.3467350	total: 30m 13s	remaining: 20m 8s
600:	learn: 0.3467340	total: 30m 15s	remaining: 20m 5s
601:	learn: 0.3467333	total: 30m 18s	remaining: 20m 2s
602:	learn: 0.3467318	total: 30m 21s	remaining: 19m 59s
603:	learn: 0.3467304	total: 30m 24s	remaining: 19m 56s
604:	learn: 0.3467300	total: 30m 27s	remaining: 19m 52s
605:	learn: 0.3467283	total: 30m 30s	remaining: 19m 49s
606:	learn: 0.3467269	total: 30m 32s	remaining: 19m 46s
607:	learn: 0.3467257	total: 30m 35s	remaining: 19m 43s
608:	learn: 0.3467250	total: 30m 38s	remaining: 19m 40s
609:	learn: 0.3467232	total: 30m 41s	remaining: 19m 37s
610:	learn: 0.3467223	total: 30m 44s	remaining: 19m 34s
611:	learn: 0.3467220	total: 30m 46s	remaining: 19m 30s
612:	learn: 0.3467219	total: 30m 49s	remaining: 19m 27s
613:	learn: 0.3467208	total: 30m 52s	remaining: 19m 24s
614:	learn: 0.3467194	total: 30m 55s	remaining: 19m 

745:	learn: 0.3465624	total: 37m 26s	remaining: 12m 44s
746:	learn: 0.3465611	total: 37m 29s	remaining: 12m 41s
747:	learn: 0.3465608	total: 37m 32s	remaining: 12m 38s
748:	learn: 0.3465606	total: 37m 35s	remaining: 12m 35s
749:	learn: 0.3465592	total: 37m 38s	remaining: 12m 32s
750:	learn: 0.3465557	total: 37m 41s	remaining: 12m 29s
751:	learn: 0.3465543	total: 37m 44s	remaining: 12m 26s
752:	learn: 0.3465534	total: 37m 47s	remaining: 12m 23s
753:	learn: 0.3465520	total: 37m 50s	remaining: 12m 20s
754:	learn: 0.3465512	total: 37m 53s	remaining: 12m 17s
755:	learn: 0.3465510	total: 37m 55s	remaining: 12m 14s
756:	learn: 0.3465490	total: 37m 58s	remaining: 12m 11s
757:	learn: 0.3465488	total: 38m 1s	remaining: 12m 8s
758:	learn: 0.3465473	total: 38m 4s	remaining: 12m 5s
759:	learn: 0.3465465	total: 38m 7s	remaining: 12m 2s
760:	learn: 0.3465461	total: 38m 9s	remaining: 11m 59s
761:	learn: 0.3465446	total: 38m 13s	remaining: 11m 56s
762:	learn: 0.3465435	total: 38m 16s	remaining: 11m 53s

894:	learn: 0.3464083	total: 45m 37s	remaining: 5m 21s
895:	learn: 0.3464067	total: 45m 41s	remaining: 5m 18s
896:	learn: 0.3464045	total: 45m 44s	remaining: 5m 15s
897:	learn: 0.3464028	total: 45m 48s	remaining: 5m 12s
898:	learn: 0.3464018	total: 45m 52s	remaining: 5m 9s
899:	learn: 0.3464000	total: 45m 56s	remaining: 5m 6s
900:	learn: 0.3463999	total: 46m	remaining: 5m 3s
901:	learn: 0.3463987	total: 46m 4s	remaining: 5m
902:	learn: 0.3463983	total: 46m 8s	remaining: 4m 57s
903:	learn: 0.3463974	total: 46m 11s	remaining: 4m 54s
904:	learn: 0.3463971	total: 46m 14s	remaining: 4m 51s
905:	learn: 0.3463969	total: 46m 19s	remaining: 4m 48s
906:	learn: 0.3463961	total: 46m 23s	remaining: 4m 45s
907:	learn: 0.3463955	total: 46m 26s	remaining: 4m 42s
908:	learn: 0.3463906	total: 46m 30s	remaining: 4m 39s
909:	learn: 0.3463889	total: 46m 34s	remaining: 4m 36s
910:	learn: 0.3463878	total: 46m 38s	remaining: 4m 33s
911:	learn: 0.3463877	total: 46m 42s	remaining: 4m 30s
912:	learn: 0.3463866	t

In [341]:
cbm.predict_proba(X_test)

array([[0.90958106, 0.09041894],
       [0.835979  , 0.164021  ],
       [0.82411073, 0.17588927],
       ...,
       [0.84580997, 0.15419003],
       [0.85529953, 0.14470047],
       [0.77695842, 0.22304158]])

In [342]:
cbm.score(X_test, y_test)

0.8986981055231213